# Code to collect Training and Testing Data
### Run cell 1 before running cell 2

In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np

# creating mediapipe objects
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.4, min_tracking_confidence=0.4, model_complexity=2)
mp_drawing = mp.solutions.drawing_utils

#defining no of frames we need to depict one action
frames_needed = 30
# directory = 'data'
#define training data array, here we will store all data array for one action
training_data = []

counter = 0

In [2]:
# function to get each frame
def getLandmarksArrayForFrame(results):
    landmarks = []
    
    for i in range(33):
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].x)
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].y)
        landmarks.append(results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].visibility)
#         landmarks.append([results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].x,
#                          results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].y,
#                          results.pose_landmarks.landmark[mp_pose.PoseLandmark(i).value].visibility])
    
    landmarks = np.array(landmarks)
    return landmarks

In [3]:
def getTrainingData(directory):
    # Reading each video and collecting data from it present in data folder
    for video in os.listdir(directory):
        print(video)
        # in this array we will add input of shape (30, 33, 3) of each video and then append it to training_data array
        frame_array = []

        cap = cv2.VideoCapture(f'{directory}/{video}')
        if not cap.isOpened():
            print('Failed to open video')

        success, frame = cap.read()
        if not success:
            print("cap read failed in getTrainingData method")
        #resizing the window
        h, w = frame.shape[0], frame.shape[1]
        print(h, w)
#         --------------------------------------------------------------------------------------------------
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)  # WINDOW_NORMAL allows resizing
        # Resize the window
        cv2.resizeWindow('frame', w//2, h//2)  # Set the desired width and height
#         --------------------------------------------------------------------------------------------------

        # computing required values
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frames // frames_needed
        frames_indexes = np.linspace(0, total_frames - 1, frames_needed, dtype=int)

        #printing the values computed for verification
        print(f'Total frames is {total_frames}')
        print(f'step is {step}')
        print(frames_indexes)
        print(f'Lenght of frames index array is {len(frames_indexes)}')
        
        getAllFrames(frames_indexes, cap, frame_array)
        
    cap.release()

In [4]:
def getAllFrames(frames_indexes, cap, frame_array):
    global counter
    frame_counter = 0
    for frame_index in frames_indexes:
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        
        success, frame = cap.read()
        if not success:
            print('failed to read frame or video ended so skipping the video')
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames_indexes[-2])
            success, frame = cap.read()
            if not success:
                print('failed to read frame or video ended so skipping the video')
                

        frame_counter += 1
        # converted to rgb because mediapipe reads images in rgb format
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
        #detecting landmarks on each frame
        results = pose.process(rgb_frame)
        
        #drawing landmarks
#             --------------------------------------------------------------------------------------------------        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#             --------------------------------------------------------------------------------------------------
        
        if results.pose_landmarks:
            frame_array.append(getLandmarksArrayForFrame(results))
        else:
            print(f"landmarks from one frame not detected, frame no is {frame_index}")
            frame_array.append(np.zeros((99), dtype=int))
#             --------------------------------------------------------------------------------------------------
        cv2.imshow("frame", frame)
#             --------------------------------------------------------------------------------------------------
        counter += 1
        if cv2.waitKey(1) == ord('q'):
            break
        print(f"Read frame {frame_index}")
    print(f'Read {frame_counter} frames')
    frame_array = np.array(frame_array)
    training_data.append(frame_array)

## Run below code to collect data

In [5]:

print("calling getTrainingData")
getTrainingData('data_falling')

# training_data = np.array(training_data)
# print(training_data.shape)
# print(training_data)
print(training_data)
print(f'Readed {counter} frames')
cv2.destroyAllWindows()

calling getTrainingData
falling_video_1.mp4
1280 720
Total frames is 50
step is 1
[ 0  1  3  5  6  8 10 11 13 15 16 18 20 21 23 25 27 28 30 32 33 35 37 38
 40 42 43 45 47 49]
Lenght of frames index array is 30
Read frame 0
Read frame 1
Read frame 3
Read frame 5
Read frame 6
Read frame 8
Read frame 10
Read frame 11
Read frame 13
Read frame 15
Read frame 16
Read frame 18
Read frame 20
Read frame 21
Read frame 23
Read frame 25
Read frame 27
Read frame 28
Read frame 30
Read frame 32
Read frame 33
Read frame 35
Read frame 37
Read frame 38
Read frame 40
Read frame 42
Read frame 43
Read frame 45
Read frame 47
Read frame 49
Read 30 frames
falling_video_10.mp4
1280 720
Total frames is 41
step is 1
[ 0  1  2  4  5  6  8  9 11 12 13 15 16 17 19 20 22 23 24 26 27 28 30 31
 33 34 35 37 38 40]
Lenght of frames index array is 30
landmarks from one frame not detected, frame no is 0
Read frame 0
Read frame 1
Read frame 2
Read frame 4
Read frame 5
Read frame 6
Read frame 8
Read frame 9
Read frame 11
Rea

KeyboardInterrupt: 

In [19]:
len(training_data)
# np_training_data = np.array(training_data)

42

In [20]:
np_training_data = np.array(training_data)

In [21]:
np_training_data.shape

(42, 30, 99)

In [22]:
np.save("walking_landmark_data.npy", np_training_data)

In [76]:
training_data.shape

AttributeError: 'list' object has no attribute 'shape'

In [2]:
load_training_data = np.load('landmark_data.npy')

In [4]:
print(load_training_data.shape)

(42, 30, 33, 3)


# Function to obtain data from a single video